In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_selection import chi2,SelectKBest

In [2]:
import os
os.getcwd()
os.chdir(r"C:\Users\Administrator\Desktop\PYTHON\DS Case Study Eko")

In [3]:
data1=pd.read_csv("baselines.csv")
data1.head(3)

,mrn,Age,sex.factor,bmi,hypoxia_ed.factor,smoke_vape,dm.factor,htn.factor,pulm___1.factor,renal___1.factor,...,symptoms___9.factor,symptoms___8.factor,symptoms___3.factor,first_cxr_results___0.factor,first_cxr_results___1.factor,first_cxr_results___2.factor,first_cxr_results___3.factor,duration_symptoms,Ed_before_order_set,event
0,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes
1,729545,78.625586,Female,27.554594,Yes,No,Yes,Yes,Unchecked,Unchecked,...,Checked,Unchecked,Checked,Unchecked,Unchecked,Checked,Unchecked,20,Yes,Yes
2,805568,70.460677,Female,34.141729,Yes,No,Yes,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,9,No,No


In [4]:
data1.shape

(1345, 27)

In [6]:
data2=pd.read_csv("lab and vitals.csv")
data2.head(3)

,name,subject,value,time_stamp
0,s_bp_noninvasive (d),89909248,NaN,2020-07-08T13:08:29Z
1,s_bp_noninvasive (d),89909248,NaN,2020-07-08T15:53:04Z
2,s_bp_noninvasive (d),89909248,NaN,2020-07-08T18:37:38Z


In [7]:
data2.shape

(708106, 4)

In [8]:
lab_data.rename(columns={'subject':'mrn'},inplace=True)

In [11]:
dataset=pd.merge(data1,lab_data,how='left',on=['mrn'])
dataset.head()

,mrn,Age,sex.factor,bmi,hypoxia_ed.factor,smoke_vape,dm.factor,htn.factor,pulm___1.factor,renal___1.factor,...,first_cxr_results___0.factor,first_cxr_results___1.factor,first_cxr_results___2.factor,first_cxr_results___3.factor,duration_symptoms,Ed_before_order_set,event,name,value,time_stamp
0,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes,s_bp_noninvasive (d),62.971715,2020-07-02T12:26:12Z
1,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes,s_bp_noninvasive (d),61.311236,2020-07-02T14:07:31Z
2,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes,s_bp_noninvasive (d),60.569907,2020-07-02T15:48:49Z
3,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes,s_bp_noninvasive (d),64.265249,2020-07-02T17:30:07Z
4,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,Unchecked,Checked,Unchecked,8,No,Yes,s_bp_noninvasive (d),60.852819,2020-07-02T19:11:26Z


In [12]:
print(dataset.shape)

(637555, 30)


In [13]:
dataset=dataset.drop_duplicates(subset=['mrn'],keep='first')
dataset.shape

(1345, 30)

In [16]:
dataset.apply(lambda x:x.isnull().sum()/len(x)*100).sort_values(ascending=False)

value                           3.866171
time_stamp                      0.000000
Age                             0.000000
sex.factor                      0.000000
bmi                             0.000000
hypoxia_ed.factor               0.000000
smoke_vape                      0.000000
dm.factor                       0.000000
htn.factor                      0.000000
pulm___1.factor                 0.000000
renal___1.factor                0.000000
renal___2.factor                0.000000
cad.factor                      0.000000
cancer                          0.000000
any_immunosuppression           0.000000
symptoms___1.factor             0.000000
symptoms___2.factor             0.000000
symptoms___10.factor            0.000000
symptoms___9.factor             0.000000
symptoms___8.factor             0.000000
symptoms___3.factor             0.000000
first_cxr_results___0.factor    0.000000
first_cxr_results___1.factor    0.000000
first_cxr_results___2.factor    0.000000
first_cxr_result

In [17]:
dataset.value=dataset['value'].fillna(dataset['value'].mean())

In [18]:
dataset['time_stamp'].apply(type)

0         <class 'str'>
630       <class 'str'>
1060      <class 'str'>
1439      <class 'str'>
1817      <class 'str'>
              ...      
635025    <class 'str'>
635681    <class 'str'>
636228    <class 'str'>
636613    <class 'str'>
637149    <class 'str'>
Name: time_stamp, Length: 1345, dtype: object

In [19]:
dataset['time_stamp']=pd.to_datetime(dataset['time_stamp'])

In [20]:
dataset['Date']=dataset['time_stamp'].dt.day
dataset['Month']=dataset['time_stamp'].dt.month
dataset['Year']=dataset['time_stamp'].dt.year

In [21]:
dataset.head(3)

,mrn,Age,sex.factor,bmi,hypoxia_ed.factor,smoke_vape,dm.factor,htn.factor,pulm___1.factor,renal___1.factor,...,first_cxr_results___3.factor,duration_symptoms,Ed_before_order_set,event,name,value,time_stamp,Date,Month,Year
0,655528,62.321679,Male,20.388645,No,No,No,Yes,Unchecked,Unchecked,...,Unchecked,8,No,Yes,s_bp_noninvasive (d),62.971715,2020-07-02 12:26:12+00:00,2,7,2020
630,729545,78.625586,Female,27.554594,Yes,No,Yes,Yes,Unchecked,Unchecked,...,Unchecked,20,Yes,Yes,s_bp_noninvasive (d),58.688958,2020-06-30 14:29:03+00:00,30,6,2020
1060,805568,70.460677,Female,34.141729,Yes,No,Yes,Yes,Unchecked,Unchecked,...,Unchecked,9,No,No,s_bp_noninvasive (d),64.848642,2020-07-07 09:38:22+00:00,7,7,2020


In [22]:
dataset.drop(['time_stamp'],axis=1,inplace=True)

In [24]:
data=dataset.to_csv("cleaned.csv",index=True)

In [23]:
best=SelectKBest(score_func=chi2,k=12)

In [ ]:
fit=best.fit()